# Task Decomposistion in Prompts

As AI language models become more advanced, they are increasingly capable of handling complex tasks. However, these models often perform better when given clear, step-by-step instructions. Task decomposition is a powerful technique that allows us to break down complex problems into smaller, more manageable subtasks. This approach not only improves the model's performance but also enhances the interpretability and reliability of the results.

The tutorial employs a step-by-step approach to demonstrate task decomposition:

1. **Problem Analysis**: We start by examining a complex task and identifying its component parts.
2. **Subtask Definition**: We define clear, manageable subtasks that collectively address the main problem.
3. **Prompt Engineering**: For each subtask, we create targeted prompts that guide the AI model.
4. **Sequential Execution**: We implement a chain of prompts, where the output of one subtask feeds into the next.
5. **Result Synthesis**: Finally, we combine the outputs from all subtasks to form a comprehensive solution.

In [1]:
! pip install langchain langchain-google-genai

      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [2]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

# Set up OpenAI API key
os.environ["GOOGLE_API_KEY"] = ''

# Initialize the language model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

def run_prompt(prompt, **kwargs):
    """Helper function to run a prompt through the language model.

    Args:
        prompt (str): The prompt template string.
        **kwargs: Keyword arguments to fill the prompt template.

    Returns:
        str: The model's response.
    """
    prompt_template = PromptTemplate(template=prompt, input_variables=list(kwargs.keys()))
    chain = prompt_template | llm
    return chain.invoke(kwargs).content

# Breaking down Complex Task

In [3]:
complex_task = """
Analyze the financial health of a company based on the following data:
- Revenue: $10 million
- Net Income: $2 million
- Total Assets: $15 million
- Total Liabilities: $7 million
- Cash Flow from Operations: $3 million
"""

decomposition_prompt = """
Break down the task of analyzing a company's financial health into 3 subtasks. For each subtask, provide a brief description of what it should accomplish.

Task: {task}

Subtasks:
1.
"""

subtasks = run_prompt(decomposition_prompt, task=complex_task)
print(subtasks)

Subtasks:

1. **Assess Profitability and Efficiency:** This subtask focuses on determining how effectively the company is generating profit and managing its resources.  It would involve calculating key profitability ratios (e.g., Net Profit Margin = Net Income / Revenue,  Gross Profit Margin, Return on Assets (ROA) = Net Income / Total Assets) and efficiency ratios (e.g., Asset Turnover).  The analysis should interpret these ratios to understand the company's pricing strategy, cost control, and overall operational efficiency.  A comparison to industry benchmarks would strengthen this analysis.

2. **Evaluate Liquidity and Solvency:** This subtask aims to determine the company's ability to meet its short-term and long-term obligations.  It would involve calculating liquidity ratios (e.g., Current Ratio = Current Assets / Current Liabilities, Quick Ratio) to assess the company's ability to pay its immediate debts.  Solvency ratios (e.g., Debt-to-Equity Ratio = Total Liabilities / Total E

# Chaining Subtasks in Prompts

In [4]:
def analyze_profitability(revenue, net_income):
    """Analyze the company's profitability.

    Args:
        revenue (float): Company's revenue.
        net_income (float): Company's net income.

    Returns:
        str: Analysis of the company's profitability.
    """
    prompt = """
    Analyze the company's profitability based on the following data:
    - Revenue: ${revenue} million
    - Net Income: ${net_income} million

    Calculate the profit margin and provide a brief analysis of the company's profitability.
    """
    return run_prompt(prompt, revenue=revenue, net_income=net_income)

def analyze_liquidity(total_assets, total_liabilities):
    """Analyze the company's liquidity.

    Args:
        total_assets (float): Company's total assets.
        total_liabilities (float): Company's total liabilities.

    Returns:
        str: Analysis of the company's liquidity.
    """
    prompt = """
    Analyze the company's liquidity based on the following data:
    - Total Assets: ${total_assets} million
    - Total Liabilities: ${total_liabilities} million

    Calculate the current ratio and provide a brief analysis of the company's liquidity.
    """
    return run_prompt(prompt, total_assets=total_assets, total_liabilities=total_liabilities)

def analyze_cash_flow(cash_flow):
    """Analyze the company's cash flow.

    Args:
        cash_flow (float): Company's cash flow from operations.

    Returns:
        str: Analysis of the company's cash flow.
    """
    prompt = """
    Analyze the company's cash flow based on the following data:
    - Cash Flow from Operations: ${cash_flow} million

    Provide a brief analysis of the company's cash flow health.
    """
    return run_prompt(prompt, cash_flow=cash_flow)

# Run the chained subtasks
profitability_analysis = analyze_profitability(10, 2)
liquidity_analysis = analyze_liquidity(15, 7)
cash_flow_analysis = analyze_cash_flow(3)

print("Profitability Analysis:\n", profitability_analysis)
print("\nLiquidity Analysis:\n", liquidity_analysis)
print("\nCash Flow Analysis:\n", cash_flow_analysis)

Profitability Analysis:
 The profit margin is calculated as:

**Profit Margin = (Net Income / Revenue) * 100%**

In this case:

Profit Margin = ($2 million / $10 million) * 100% = 20%

**Analysis:**

The company has a profit margin of 20%, which indicates that for every dollar of revenue generated, the company retains $0.20 as net income after all expenses are deducted.  A 20% profit margin is generally considered good, especially compared to many industries.  However, a more thorough analysis would require comparing this margin to:

* **Industry benchmarks:**  A 20% margin might be excellent in one industry but average or poor in another.  Comparing to competitors is crucial.
* **Past performance:**  Is this margin an improvement, a decline, or consistent with previous years?  Trends are important for understanding profitability.
* **Cost structure:**  A deeper dive into the company's expenses is needed to understand *why* the margin is 20%.  Are costs well-managed, or is there room f

# Integrating the Results

In [5]:
def integrate_results(profitability, liquidity, cash_flow):
    """Integrate the results from subtasks to provide an overall analysis.

    Args:
        profitability (str): Profitability analysis.
        liquidity (str): Liquidity analysis.
        cash_flow (str): Cash flow analysis.

    Returns:
        str: Overall analysis of the company's financial health.
    """
    prompt = """
    Based on the following analyses, provide an overall assessment of the company's financial health:

    Profitability Analysis:
    {profitability}

    Liquidity Analysis:
    {liquidity}

    Cash Flow Analysis:
    {cash_flow}

    Summarize the key points and give an overall evaluation of the company's financial position.
    """
    return run_prompt(prompt, profitability=profitability, liquidity=liquidity, cash_flow=cash_flow)

overall_analysis = integrate_results(profitability_analysis, liquidity_analysis, cash_flow_analysis)
print("Overall Financial Health Analysis:\n", overall_analysis)

Overall Financial Health Analysis:
 The company shows some positive signs of financial health, but a conclusive assessment is hampered by incomplete data.

**Positive Indicators:**

* **Profitable Operations:** A 20% profit margin suggests strong profitability, indicating efficient operations and good pricing strategies.  However, this needs benchmarking against industry averages and past performance for a true evaluation.
* **Positive Operating Cash Flow:**  $3 million in positive cash flow from operations is a significant positive, showing the company is generating cash from its core business activities and covering its expenses.  However, without data on investing and financing activities, and the time period covered, this assessment is incomplete.


**Missing Information and Concerns:**

* **Liquidity:**  The lack of current asset and liability data prevents a proper assessment of the company's short-term solvency.  A crucial current ratio calculation is impossible without this inf